# Collision Detection and Visualization Demo

This notebook demonstrates collision detection capabilities and distance visualization using the LinkMotion library.

## Overview
This example showcases:
1. **Robot-Environment Collision Detection**: Detecting collisions between robot parts and obstacles
2. **Distance Visualization**: Showing closest points between objects
3. **Interactive Movement**: Moving joints and observing collision changes in real-time

## Key Components
- `CollisionManager`: Handles collision detection between robot links and environment
- `CollisionVisualizer`: Renders collision information and distance points
- `MoveManager`: Controls robot joint movements
- `MoveVisualizer`: Displays robot configurations

## Scenario
The robot includes a wall obstacle, and we demonstrate how the robot's leg can approach and potentially collide with the wall as joints are moved.

Let's start by importing the required libraries:

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

from linkmotion.visual import MoveVisualizer, CollisionVisualizer
from linkmotion import (
    Robot,
    Link,
    Joint,
    JointType,
    Transform,
    MoveManager,
    CollisionManager,
)

## Robot and Environment Setup

Creating a humanoid robot with a wall obstacle for collision testing:

In [ ]:
humanoid = Robot()

body = Link.from_box("body", extents=np.array((3, 2, 10)))

t = Transform(
    rotate=R.from_rotvec(90.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((4, 0, 3)),
)
right_arm = Link.from_cylinder("right_arm", radius=0.5, height=5, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(90.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((-4, 0, 3)),
)
left_arm = Link.from_cylinder("left_arm", radius=0.5, height=5, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(180.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((1, 0, -8)),
)
right_leg = Link.from_cone("right_leg", radius=0.5, height=6, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(180.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((-1, 0, -8)),
)
left_leg = Link.from_cone("left_leg", radius=0.5, height=6, default_transform=t)

t = Transform(translate=np.array((0, 0, -5)))
head = Link.from_sphere("head", 3, center=np.array((0, 0, 8)))

left_leg_joint = Joint(
    "left_leg_joint",
    JointType.REVOLUTE,
    child_link_name="left_leg",
    parent_link_name="body",
    center=np.array((-1, 0, -5)),
    direction=np.array((1, 0, 0)),
    min_=-np.pi / 2.0,
    max_=np.pi / 4.0,
)

t = Transform(translate=np.array((0, -5, 0)))
wall = Link.from_box(
    "wall",
    extents=np.array((20, 1, 20)),
    default_transform=t,
    color=np.array((0.0, 0.8, 0.8, 0.2)),
)

humanoid.add_link(body)
humanoid.add_link(right_arm)
humanoid.add_link(left_arm)
humanoid.add_link(right_leg)
humanoid.add_link(left_leg)
humanoid.add_link(head)
humanoid.add_joint(left_leg_joint)
humanoid.add_link(wall)

mm = MoveManager(humanoid)
cm = CollisionManager(mm)

## Initial Configuration - Distance Visualization

Showing the distance between the left leg and wall in the default robot configuration:

In [ ]:
plot = MoveVisualizer.robot(mm)
plot = CollisionVisualizer.distance_points(
    cm, {"left_leg"}, {"wall"}, plot=plot, point_size=0.5, width=0.2
)
plot.display()

## Output Example
![OutputExample](./img/collision/collision1.png)

## Moderate Joint Movement

Moving the left leg joint to bring it closer to the wall:

In [ ]:
mm.move(joint_name="left_leg_joint", value=-np.pi / 6.0)
plot = MoveVisualizer.robot(mm)
plot = CollisionVisualizer.distance_points(
    cm, {"left_leg"}, {"wall"}, plot=plot, point_size=0.5, width=0.2
)
plot.display()

## Output Example
![OutputExample](./img/collision/collision2.png)

## Maximum Joint Movement - Potential Collision

Moving the left leg joint to its extreme position, showing potential collision:

In [ ]:
mm.move(joint_name="left_leg_joint", value=-np.pi / 2.0)
plot = MoveVisualizer.robot(mm)
plot = CollisionVisualizer.distance_points(
    cm, {"left_leg"}, {"wall"}, plot=plot, point_size=0.5, width=0.2
)
plot.display()

## Output Example
![OutputExample](./img/collision/collision3.png)